In [8]:
import requests
from datetime import datetime

from pprint import pprint
from dateutil.parser import parse
from datetime import datetime
import jmespath

from jinja2 import Environment, FileSystemLoader
from email.mime.text import MIMEText
import sys
# sys.setdefaultencoding('utf-8')

In [9]:
# !pip install jmespath 

In [69]:
class Info:
    def __init__(self,symbol):
        self.token = 'pk_4667974e324a475cb0c4eabb796d2088'
        self.url = 'https://cloud.iexapis.com/v1/stock/'
        self.symbol = symbol.lower()
        self.data = self.get_data()
#         return 
    
    def get_data(self):
      
        URL='{}{}/batch?types=quote,news&range=1m&last=1&token={}'.format(self.url, self.symbol, self.token)
        response = requests.get(URL)
        if response.status_code == 200:
            return response.json()
        else:
            print('ERROR',response.status_code)
            
    def get_news(self) :
        return jmespath.search('news[0].summary', self.data)#.encode('utf-8')#.decode("utf-8", "replace")
#         return self.data['news'][0]['summary']
        
    def get_source(self):
        return jmespath.search('news[0].source', self.data).encode('utf-8')
#         return self.data['news'][0]['source']
    def get_headline(self):
        return jmespath.search('news[0].headline', self.data).encode('utf-8')

        
    def get_datetime(self):
        ## https://stackoverflow.com/questions/10286224/javascript-timestamp-to-python-datetime-conversion
        return (datetime
                .utcfromtimestamp(
                    jmespath.search('news[0].datetime', self.data)/1000
                )
                .strftime('%Y-%m-%d %H:%M'))
    
    def companyName(self):
        return jmespath.search('quote.companyName', self.data)#.encode('utf-8')
    
    def price(self):
#         close = jmespath.search('quote.clsoe', self.data)
        d = []
        for key in ['close','change', 'high', 'low', 'avgTotalVolume']:
            d.append( "{}: {}".format(key, self.data['quote'][key]) )
        return ' '.join(map(str,d))
                               

url = 'https://cloud.iexapis.com/v1/stock/'
stock = 'aapl'

info = Info(stock)
# info.get_data()
# print(info.get_new())b


In [70]:
class Builder:
    ## Jinja2 Template builder 
    def __init__(self):
        return 
    
    def build_template(self, symbol, companyNames,  news, prices):
        try:
            ## Load the html tempate from template/email.html
            file_loader = FileSystemLoader('template')
            env = Environment(loader=file_loader)
            template =  env.get_template('email.html')
            
            ## Build Jinja2 template
            output = template.render(symbols=symbol, companyNames=companyNames,  news=news,prices=prices)
#             print(output)
            body = MIMEText(output, "html")
            return output
        except Exception as e:
            print('ERROR', e)
            print()
        
            

In [74]:
STOCKS = ['AAPL','FSLY', 'MSFT']
companyNames = []
news = []
prices = []

for stock in STOCKS:
    stock_data = Info(stock)
    companyNames.append(stock_data.companyName())
    news.append(stock_data.get_news())
    prices.append(stock_data.price())


'close: 444.45 change: -11.16 high: 454.7 low: 441.17 avgTotalVolume: 37499927'

In [75]:
# companyNames = [x.encode('utf-8') for x in companyNames]
# news = [x.encode('utf-8') for x in news]
# news

In [79]:
builder = Builder()
report = builder.build_template(STOCKS, companyNames, news, prices)
# print(report)

# print(report)
with open('stock_report.html', 'w') as fileout:
    fileout.write(report)